In [2]:
import pandas as pd
import numpy as np

# Read excel
df = pd.read_excel('./dataset/Testing PTR Rilis B Oktober 2024.xlsx', skiprows=9, sheet_name='Skenario Regresi PTR (Tim UAT)')
# Drop kolom No
df.drop(columns='No', index=1, inplace=True)
# Isi NaN values di kolom Features jadi copy paste nilai pertama di atas
df['Features'] = df['Features'].ffill()
# Rename kolom
df.rename(columns={'Sub Fitur': 'Sub-features', 'Rekening Sumber\n[Jika ada]' : 'Rekening Sumber', 'Data yang Digunakan\n[Jika ada]' : 'Data yang digunakan',
                   'FT\n[Jika Ada]' : 'FT', 'Tanggal Eksekusi\n[harus diisi]' : 'Eksekusi Android', 'Tanggal Passed\n[harus diisi]' : 'Passed Android',
                   'Tanggal Eksekusi\n[harus diisi].1' : 'Eksekusi iOS', 'Tanggal Passed\n[harus diisi].1' : 'Passed iOS'}, inplace=True)



FileNotFoundError: [Errno 2] No such file or directory: './dataset/Testing PTR Rilis B Oktober 2024.xlsx'

In [2]:
# Drop all columns that contain 'Unnamed' in their name
df = df.drop(df.filter(like='Unnamed').columns, axis=1)
print(df.columns)


Index(['Features', 'Sub-features', 'Rekening Sumber', 'Data yang digunakan',
       'Tester', 'PO', 'Tipe Device HP', 'FT', 'Telko provider HP',
       'Status Android', 'Status IOS', 'Link Report Test',
       'Description Issue dan Evidence', 'Eksekusi Android', 'Passed Android',
       'Eksekusi iOS', 'Passed iOS', 'Komentar'],
      dtype='object')


In [3]:
df = df[[
    'Features', 'Sub-features', 'Rekening Sumber', 'Data yang digunakan',
       'Tester', 'PO', 'Tipe Device HP', 'FT', 'Telko provider HP',
       'Status Android', 'Status IOS', 'Link Report Test',
       'Description Issue dan Evidence', 'Eksekusi Android', 'Eksekusi iOS',
       'Passed Android', 'Passed iOS', 'Komentar'

]]

In [5]:
df.head(10)

,Features,Sub-features,Rekening Sumber,Data yang digunakan,Tester,PO,Tipe Device HP,FT,Telko provider HP,Status Android,Status IOS,Link Report Test,Description Issue dan Evidence,Eksekusi Android,Eksekusi iOS,Passed Android,Passed iOS,Komentar
0,New Aktivasi (With Debit Card),Aktivasi Via FR,Android\n\niOS\n,Android\n\niOS\n,Rezki - Hansen,NaN,Android\n\niOS\niPhone 12 Pro Max (iOS 17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
2,New Aktivasi (With Debit Card),Login Other Device Lupa Password,Android\n7216306729\niOS\n7216434468,Android\n\niOS\n,Nabila - Kintan,NaN,Android\nSamsung Z Flip 4\niOS\niPhone 13 Pro Max,Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
3,New Aktivasi (With Debit Card),Login Other Device Lupa PIN,Android\n7216306729\niOS\n7216434468,Android\n\niOS\n,Nabila - Kintan,NaN,Android\nSamsung Z Flip 4\niOS\niPhone 13 Pro Max,Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
4,New Aktivasi (With Debit Card),Login Other Device Dengan Akun Terblokir,Android\n7216306729iOS\n7216434468,Android\n\niOS\n,Nabila - Kintan,NaN,Android\nSamsung Z Flip 4\niOS\niPhone 13 Pro Max,Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
5,Lupa Password & PIN,Lupa Password dari menu profil,Android\n7269588572 iOS\n7283171579,Android\n\niOS\n,Qisty - Tiwi,NaN,Android\nNothing phone 1\niOS\nIphone 13 (17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
6,Lupa Password & PIN,Lupa Password Dari Prelogin,Android\n7269588572\niOS\n7283171579,Android\n\niOS\n,Qisty - Tiwi,NaN,Android\nNothing phone 1\niOS\nIphone 13 (17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
7,Lupa Password & PIN,Lupa PIN,Android\n7269588572\niOS\n7283171579,Android\n\niOS\n,Qisty - Tiwi,NaN,Android\nNothing phone 1\niOS\nIphone 13 (17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
8,Lupa Password & PIN,Lupa Password dari menu profil Kemudian lupa PIN,Android\n7269588572\niOS\n7283171579,Android\n\niOS\n,Qisty - Tiwi,NaN,Android\nNothing phone 1\niOS\nIphone 13 (17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
9,Lupa Password & PIN,Lupa PIN Kemudian Lupa Password,Android\n7269588572\niOS\n7283171579,Android\n\niOS\n,Qisty - Tiwi,NaN,Android\nNothing phone 1\niOS\nIphone 13 (17.6.1),Android\n\niOS\n,Android\n\niOS\n,Passed,Passed,Android\n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN
10,QRIS (FDS),MPM OFF Us (Payment),Android\n7200545588\n\niOS\n7235141269,Android\n081293394417\niOS\n082219434403,Andi - Owan,NaN,Android : Redmi Note 12 Pro - OS Android Versi...,Android | FDS\n(1). Rp. 1 + tip 1 Rp = FT24284...,Android : Simpati Telkomsel\n\niOS : Simpati ...,Passed,Passed,Android \n\niOS\n,Android\n\niOS\n,2024-10-09,2024-10-09,2024-10-09,2024-10-09,NaN


### Extracting Rekening Sumber for iOS and Android (old way)

In [6]:
import re
import numpy as np
import pandas as pd

def extracted_ios_android(df, column_name, android_column_name='Android', ios_column_name='iOS'):
    """
    Extract Android and iOS rekenings or any other relevant information from a specified column.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    column_name (str): The name of the column to extract data from.
    android_column_name (str): The name of the column to store extracted Android data.
    ios_column_name (str): The name of the column to store extracted iOS data.
    
    Returns:
    pd.DataFrame: A dataframe with two new columns: for Android and iOS data.
    """
    
    # Clean the specified column to remove excess spaces and newlines
    df[column_name] = df[column_name].str.replace(r'\n+', ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()

    # Function to extract Android and iOS values
    def extract_android_ios(row):
        android_values = np.nan
        ios_values = np.nan
        
        if isinstance(row, str):
            # Extract Android rekenings (both Rekening 1 and Rekening 2)
            android_match = re.findall(r'Android[: ]+Rekening \d+ ?: ?(\d+)', row, re.IGNORECASE)
            if android_match:
                android_values = ', '.join(android_match)  # Joining multiple values as a single string
            
            # Fallback for single Android number if no 'Rekening' is specified
            if not android_match:
                single_android_match = re.search(r'Android[: ]*(\d+)', row, re.IGNORECASE)
                if single_android_match:
                    android_values = single_android_match.group(1)
            
            # Extract iOS rekenings (both Rekening 1 and Rekening 2)
            ios_match = re.findall(r'iOS[: ]+Rekening \d+ ?: ?(\d+)', row, re.IGNORECASE)
            if ios_match:
                ios_values = ', '.join(ios_match)
            
            # Fallback for single iOS number if no 'Rekening' is specified
            if not ios_match:
                single_ios_match = re.search(r'iOS[: ]*(\d+)', row, re.IGNORECASE)
                if single_ios_match:
                    ios_values = single_ios_match.group(1)
        
        return pd.Series([android_values, ios_values], index=[android_column_name, ios_column_name])

    # Apply the extraction function to the specified column
    df[[android_column_name, ios_column_name]] = df[column_name].apply(extract_android_ios)
    
    return df

# Example of usage:
# df = pd.read_excel('your_file.xlsx')
df = extracted_ios_android(df, 'Rekening Sumber', 'Android Rekening', 'iOS Rekening')

# Display the result
print(df[['Rekening Sumber', 'Android Rekening', 'iOS Rekening']])


                       Rekening Sumber Android Rekening iOS Rekening
0                          Android iOS              NaN          NaN
2    Android 7216306729 iOS 7216434468       7216306729   7216434468
3    Android 7216306729 iOS 7216434468       7216306729   7216434468
4     Android 7216306729iOS 7216434468       7216306729   7216434468
5    Android 7269588572 iOS 7283171579       7269588572   7283171579
..                                 ...              ...          ...
163  Android 7200545588 iOS 7235141269       7200545588   7235141269
164             Android iOS 7126971178              NaN   7126971178
165                        Android iOS              NaN          NaN
166                        Android iOS              NaN          NaN
167                        Android iOS              NaN          NaN

[167 rows x 3 columns]


### Extracting Rekening Sumber, Device HP for Android and iOS (New)

Extracting all the values after android and iOS text, including text and digits (more global function)

In [7]:
import re

def extracted_ios_android(df, column_name, android_column_name, ios_column_name):
    """
    Extract Android and iOS from the specified column.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    column_name (str): The name of the column to extract data from.
    android_column_name (str): The name of the column to store extracted Android data.
    ios_column_name (str): The name of the column to store extracted iOS data.

    Returns:
    pd.DataFrame: A dataframe with two new columns for Android and iOS data.
    """
    
    # Clean the specified column to remove excess spaces and newlines
    df[column_name] = df[column_name].str.replace(r'\n+', ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    # Function to extract Android and iOS devices
    def extract_android_ios(row):
        android_values = np.nan
        ios_values = np.nan
        
        if isinstance(row, str):
            # Extract Android device
            android_match = re.search(r'Android\s*[:]*\s*(.*?)\s*(?=iOS|$)', row, re.IGNORECASE)
            if android_match:
                android_values = android_match.group(1).strip() or np.nan  # Return NaN if blank
            
            # Extract iOS device
            ios_match = re.search(r'iOS\s*[:]*\s*(.*)', row, re.IGNORECASE)
            if ios_match:
                ios_values = ios_match.group(1).strip() or np.nan  # Return NaN if blank
                
        return pd.Series([android_values, ios_values], index=[android_column_name, ios_column_name])
    
    # Apply the extraction function to the specified column
    df[[android_column_name, ios_column_name]] = df[column_name].apply(extract_android_ios)
    
    return df


In [8]:
df = extracted_ios_android(df, 'Rekening Sumber', 'Rek_Android', 'Rek_iOS')
df = extracted_ios_android(df, 'Tipe Device HP', 'HP_Android', 'HP_iOS')
df = extracted_ios_android(df, 'Telko provider HP', 'Provider_Android', 'Provider_iOS')

In [9]:
df.loc[35]['FT'] == df.loc[36]['FT']

False

In [10]:
# Replace NaN values in row 36 with values from row 35, but keep non-NaN values in row 36 (case : 2 rekenings at the same row, so we need to manually seperate it, and then 
# duplicate values to fill the NaN)
df.loc[36] = df.loc[36].fillna(df.loc[35])
df.loc[[35,36]]

,Features,Sub-features,Rekening Sumber,Data yang digunakan,Tester,PO,Tipe Device HP,FT,Telko provider HP,Status Android,...,Passed iOS,Komentar,Android Rekening,iOS Rekening,Rek_Android,Rek_iOS,HP_Android,HP_iOS,Provider_Android,Provider_iOS
35,Change password (FDS),1. Siapkan 2 rekening dengan CIF yang sama \n2...,Android : 7243447776 iOS,Android\n\niOS\n,Arul - Fira,NaN,Android : OPPO Find N2 Flip iOS,"Android :\n[FT2428BYV31, FT24284QOCTW]\n\niOS\n",Android iOS,Passed,...,2024-10-11,Android : Change Password 10.06 WIB\n\nIOS\n15.48,7243447776,NaN,7243447776,NaN,OPPO Find N2 Flip,NaN,NaN,NaN
36,Change password (FDS),1. Siapkan 2 rekening dengan CIF yang sama \n2...,Android : 7280252683 iOS,Android\n\niOS\n,Arul - Fira,NaN,Android : OPPO Find N2 Flip iOS,"Android :\n[FT24284DB6NS, FT24284YNNOO]\n\niOS\n",Android iOS,Passed,...,2024-10-11,Android : Change Password 10.06 WIB\n\nIOS\n15.48,7280252683,NaN,7280252683,NaN,OPPO Find N2 Flip,NaN,NaN,NaN


In [11]:
# set multiindex
d = {'Rek_Android': 'Rekening Sumber', 'Rek_iOS': 'Rekening Sumber', 
     'HP_Android': 'Tipe Device HP', 'HP_iOS': 'Tipe Device HP',
     'Provider_Android': 'Telko Provider HP', 'Provider_iOS': 'Telko Provider HP',
     'Eksekusi Android': 'Tanggal Eksekusi', 'Eksekusi iOS': 'Tanggal Eksekusi',
     'Passed Android': 'Tanggal Passed', 'Passed iOS' : 'Tanggal Passed'
     }
df.columns = pd.MultiIndex.from_tuples([*zip(map(d.get, df), df)])

# set index jadi features dan Sub-features
# df.set_index([(np.nan, 'Features'), (np.nan, 'Sub-features')], inplace=True)

# Replace all blank android and ios to NaN values
df[np.nan, 'Data yang digunakan'] = df[np.nan, 'Data yang digunakan'].replace('Android\n\niOS\n', np.nan)

# Replace all provider


In [12]:
# Grabbing and fixing all values that contain any TSEL or Simpati Telkomsel values, both in Android and iOS

df[('Telko Provider HP', 'Provider_iOS')] = df[('Telko Provider HP', 'Provider_iOS')].replace({
    'TSEL': 'Telkomsel',
    'Simpati Telkomsel': 'Telkomsel'})

df[('Telko Provider HP', 'Provider_Android')] = df[('Telko Provider HP', 'Provider_Android')].replace({
    'TSEL': 'Telkomsel',
    'Simpati Telkomsel': 'Telkomsel'})

In [13]:
df['Telko Provider HP', 'Provider_iOS'].value_counts()

(Telko Provider HP, Provider_iOS)
XL           22
Telkomsel    13
tri           5
Name: count, dtype: int64

In [14]:
df.drop(columns=[(np.nan, 'Rekening Sumber'), (np.nan, 'Tipe Device HP'), (np.nan, 'Telko provider HP')], axis=1, inplace=True)

In [15]:
df.columns

MultiIndex([(                nan,                       'Features'),
            (                nan,                   'Sub-features'),
            (                nan,            'Data yang digunakan'),
            (                nan,                         'Tester'),
            (                nan,                             'PO'),
            (                nan,                             'FT'),
            (                nan,                 'Status Android'),
            (                nan,                     'Status IOS'),
            (                nan,               'Link Report Test'),
            (                nan, 'Description Issue dan Evidence'),
            ( 'Tanggal Eksekusi',               'Eksekusi Android'),
            ( 'Tanggal Eksekusi',                   'Eksekusi iOS'),
            (   'Tanggal Passed',                 'Passed Android'),
            (   'Tanggal Passed',                     'Passed iOS'),
            (                nan, 

In [16]:
 df.reset_index(drop=True, inplace=True)

In [17]:
df

NaN                                            \
                           Features                              Sub-features   
0    New Aktivasi (With Debit Card)                           Aktivasi Via FR   
1    New Aktivasi (With Debit Card)          Login Other Device Lupa Password   
2    New Aktivasi (With Debit Card)               Login Other Device Lupa PIN   
3    New Aktivasi (With Debit Card)  Login Other Device Dengan Akun Terblokir   
4               Lupa Password & PIN            Lupa Password dari menu profil   
..                              ...                                       ...   
162         Tarik Tunai Tanpa Kartu        ATM BSI (iOS)\nALfa Midi (Android)   
163                Korlantas Signal                    Korlantas Signal (Inq)   
164                   Product Offer                        List Product Offer   
165                   Product Offer                     Routing Product Offer   
166                      Byond Lyfe                                Byond Lyfe   

                                                                     \
                          Data yang digunakan           Tester   PO   
0                                         NaN   Rezki - Hansen  NaN   
1                                         NaN  Nabila - Kintan  NaN   
2                                         NaN  Nabila - Kintan  NaN   
3                                         NaN  Nabila - Kintan  NaN   
4                                         NaN     Qisty - Tiwi  NaN   
..                                        ...              ...  ...   
162  Android\n081293394417\niOS\n082219434403      Andi - Owan  NaN   
163                                       NaN        Aga- Ajul  NaN   
164                                       NaN   Rezki - Hansen  NaN   
165                                       NaN   Rezki - Hansen  NaN   
166                                       NaN  Prita - Natasya  NaN   

                                                   \
                     FT Status Android Status IOS   
0      Android\n\niOS\n         Passed     Passed   
1      Android\n\niOS\n         Passed     Passed   
2      Android\n\niOS\n         Passed     Passed   
3      Android\n\niOS\n         Passed     Passed   
4      Android\n\niOS\n         Passed     Passed   
..                  ...            ...        ...   
162  Android\n\n\niOS\n         Failed     Failed   
163  Android\n\niOS :\n         Passed     Passed   
164    Android\n\niOS\n         Passed     Passed   
165    Android\n\niOS\n         Passed     Passed   
166    Android\n\niOS\n         Failed     Failed   

                                                        \
                                      Link Report Test   
0                                     Android\n\niOS\n   
1                                     Android\n\niOS\n   
2                                     Android\n\niOS\n   
3                                     Android\n\niOS\n   
4                                     Android\n\niOS\n   
..                                                 ...   
162  Android\n\niOS\n- icon bsi (atas Bawah) & sosm...   
163                                   Android\n\niOS\n   
164                                   Android\n\niOS\n   
165                                   Android\n\niOS\n   
166  Android\nhttps://istqa.atlassian.net/browse/PS...   

                                                        ... Tanggal Passed  \
                        Description Issue dan Evidence  ...     Passed iOS   
0                                     Android\n\niOS\n  ...     2024-10-09   
1                                     Android\n\niOS\n  ...     2024-10-09   
2                                     Android\n\niOS\n  ...     2024-10-09   
3                                     Android\n\niOS\n  ...     2024-10-09   
4                                     Android\n\niOS\n  ...     2024-10-09   
..                                                 ...  ...    

### Just in case kalo butuh list multilevel

In [295]:
columns_multilevel = [
    ('Features', ''),
    ('Sub-features', ''),
    ('Rekening Sumber', ''),
    ('Rekening Sumber', 'Rek_Android'),
    ('Rekening Sumber', 'Rek_iOS'),
    ('Data yang digunakan', ''),
    ('Tester', ''),
    ('PO', ''),
    ('Tipe Device HP', ''),
    ('Tipe Device HP', 'HP_Android'),
    ('Tipe Device HP', 'HP_iOS'),
    ('FT', ''),
    ('Telco provider HP', ''),
    ('Status Android', ''),
    ('Status iOS', ''),
    ('Link Report Test', ''),
    ('Description Issue dan Evidence', ''),
    ('Tanggal Eksekusi', 'Eksekusi Android'),
    ('Tanggal Eksekusi', 'Eksekusi iOS'),
    ('Tanggal Passed', 'Passed Android'),
    ('Tanggal Passed', 'Passed iOS'),
    ('Komentar', '')
]

# Set the new multi-level columns to the DataFrame
df.columns = pd.MultiIndex.from_tuples(columns_multilevel)

# Display the DataFrame
print(df)


                           Features                              Sub-features  \
                                                                                
0    New Aktivasi (With Debit Card)                           Aktivasi Via FR   
2    New Aktivasi (With Debit Card)          Login Other Device Lupa Password   
3    New Aktivasi (With Debit Card)               Login Other Device Lupa PIN   
4    New Aktivasi (With Debit Card)  Login Other Device Dengan Akun Terblokir   
5               Lupa Password & PIN            Lupa Password dari menu profil   
..                              ...                                       ...   
163         Tarik Tunai Tanpa Kartu        ATM BSI (iOS)\nALfa Midi (Android)   
164                Korlantas Signal                    Korlantas Signal (Inq)   
165                   Product Offer                        List Product Offer   
166                   Product Offer                     Routing Product Offer   
167                      Byo